## 一、数据准备

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',200)
pd.set_option('expand_frame_repr',False)

In [3]:
train_df=pd.read_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\train_afterCleaning.csv')
test_df=pd.read_csv(r'D:\文件\学习\数据集\【天池】贷款违约预测\test_afterCleaning.csv')

### 字段说明(Field Description)

<br>id	为贷款清单分配的唯一信用证标识
<br>loanAmnt	贷款金额
<br>term	贷款期限（year）
<br>interestRate	贷款利率
<br>installment	分期付款金额
<br>grade	贷款等级
<br>subGrade	贷款等级之子级
<br>employmentTitle	就业职称
<br>employmentLength	就业年限（年）
<br>homeOwnership	借款人在登记时提供的房屋所有权状况
<br>annualIncome	年收入
<br>verificationStatus	验证状态
<br>issueDate	贷款发放的月份
<br>purpose	借款人在贷款申请时的贷款用途类别
<br>postCode	借款人在贷款申请中提供的邮政编码的前3位数字
<br>regionCode	地区编码
<br>dti	债务收入比
<br>delinquency_2years	借款人过去2年信用档案中逾期30天以上的违约事件数
<br>ficoRangeLow	借款人在贷款发放时的fico所属的下限范围
<br>ficoRangeHigh	借款人在贷款发放时的fico所属的上限范围
<br>openAcc	借款人信用档案中未结信用额度的数量
<br>pubRec	贬损公共记录的数量
<br>pubRecBankruptcies	公开记录清除的数量
<br>revolBal	信贷周转余额合计
<br>revolUtil	循环额度利用率，或借款人使用的相对于所有可用循环信贷的信贷金额
<br>totalAcc	借款人信用档案中当前的信用额度总数
<br>initialListStatus	贷款的初始列表状态
<br>applicationType	表明贷款是个人申请还是与两个共同借款人的联合申请
<br>earliesCreditLine	借款人最早报告的信用额度开立的月份
<br>title	借款人提供的贷款名称
<br>policyCode	公开可用的策略_代码=1新产品不公开可用的策略_代码=2
<br>n系列匿名特征	匿名特征n0-n14，为一些贷款人行为计数特征的处理m

In [4]:
train_df=train_df.drop('Unnamed: 0',axis=1)
test_df=test_df.drop('Unnamed: 0',axis=1)

In [5]:
label=train_df['isDefault']
train_data=train_df.drop('isDefault',axis=1)
test_data=test_df.copy()

In [6]:
train_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month
0,0,35000.0,5,19.52,917.97,4,21,320.0,2,2,110000.0,2,"['2014', '07', '01']",1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,"['Aug', '2001']",1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,2001,Aug
1,1,18000.0,5,18.49,461.90,3,16,219843.0,5,0,46000.0,2,"['2012', '08', '01']",0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,"['May', '2002']",1723.0,1.0,0.0,3.0,4.0,4.0,10.0,5.0,4.0,6.0,11.0,4.0,13.0,0.0,0.0,0.0,1.0,2012,8,1,2002,May
2,2,12000.0,5,16.99,298.17,3,17,31698.0,8,0,74000.0,2,"['2015', '10', '01']",0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,"['May', '2006']",0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,1,2006,May
3,3,11000.0,3,7.26,340.96,0,3,46854.0,10,1,118000.0,1,"['2015', '08', '01']",4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,"['May', '1999']",4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,1,1999,May
4,4,3000.0,3,12.99,101.07,2,11,54.0,10,1,29000.0,2,"['2016', '03', '01']",10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,"['Aug', '1977']",11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,2016,3,1,1977,Aug


In [7]:
test_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month
0,800000,14000.0,3,10.99,458.28,1,7,7027.0,10,0,80000.0,0,"['2014', '07', '01']",0,163.0,21,10.56,1.0,715.0,719.0,17.0,0.0,0.0,9846.0,30.7,29.0,0,0,"['Nov', '1974']",0.0,1.0,1.0,4.0,6.0,6.0,6.0,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0,2014,7,1,1974,Nov
1,800001,20000.0,5,14.65,472.14,2,14,60426.0,10,0,50000.0,0,"['2015', '07', '01']",2,235.0,8,21.40,2.0,670.0,674.0,5.0,0.0,0.0,8946.0,56.6,14.0,0,0,"['Jul', '2001']",5.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0,2015,7,1,2001,Jul
2,800002,12000.0,3,19.99,445.91,3,18,23547.0,2,1,60000.0,2,"['2016', '10', '01']",0,526.0,20,33.50,0.0,710.0,714.0,12.0,0.0,0.0,970.0,17.6,43.0,1,0,"['Aug', '2006']",0.0,1.0,0.0,1.0,4.0,4.0,1.0,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0,2016,10,1,2006,Aug
3,800003,17500.0,5,14.31,410.02,2,13,636.0,4,0,37000.0,1,"['2014', '11', '01']",4,248.0,11,13.95,0.0,685.0,689.0,10.0,1.0,1.0,10249.0,52.3,18.0,0,0,"['Jul', '2002']",4.0,1.0,0.0,2.0,2.0,2.0,4.0,7.0,2.0,8.0,14.0,2.0,10.0,0.0,0.0,0.0,3.0,2014,11,1,2002,Jul
4,800004,35000.0,3,17.09,1249.42,3,15,368446.0,0,1,80000.0,1,"['2017', '10', '01']",0,115.0,8,24.97,0.0,685.0,689.0,19.0,0.0,0.0,33199.0,35.6,22.0,0,0,"['Dec', '2000']",0.0,1.0,0.0,8.0,11.0,11.0,9.0,11.0,3.0,16.0,18.0,11.0,19.0,0.0,0.0,0.0,1.0,2017,10,1,2000,Dec


## 二、特征工程

In [8]:
#创建同时删除训练集和测试集的无用字段函数
def drop_column(train_df,test_df,col_name):
    train_df=train_df.drop(col_name,axis=1)
    test_df=test_df.drop(col_name,axis=1)
    return train_df,test_df

In [9]:
train_data,test_data=drop_column(train_data,test_data,'issueDate')
train_data,test_data=drop_column(train_data,test_data,'earliesCreditLine')

In [10]:
train_data['earliesCreditLine_month'].value_counts()

Aug    79208
Sep    77698
Oct    76726
Nov    70407
Mar    65035
Dec    64493
Jul    64440
May    62440
Jun    61858
Apr    61015
Jan    58605
Feb    58075
Name: earliesCreditLine_month, dtype: int64

In [11]:
#对earliesCreditLine_month做映射
def month_transform(df):
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Jan','1')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Feb','2')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Mar','3')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Apr','4')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'May','5')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Jun','6')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Jul','7')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Aug','8')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Sep','9')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Oct','10')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Nov','11')
    df['earliesCreditLine_month']=df['earliesCreditLine_month'].str.replace(
        'Dec','12')
    return df

In [12]:
#调用日期映射函数
train_data=month_transform(train_data)
test_data=month_transform(test_data)

In [13]:
train_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month
0,0,35000.0,5,19.52,917.97,4,21,320.0,2,2,110000.0,2,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,2001,8
1,1,18000.0,5,18.49,461.90,3,16,219843.0,5,0,46000.0,2,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,1723.0,1.0,0.0,3.0,4.0,4.0,10.0,5.0,4.0,6.0,11.0,4.0,13.0,0.0,0.0,0.0,1.0,2012,8,1,2002,5
2,2,12000.0,5,16.99,298.17,3,17,31698.0,8,0,74000.0,2,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,1,2006,5
3,3,11000.0,3,7.26,340.96,0,3,46854.0,10,1,118000.0,1,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,1,1999,5
4,4,3000.0,3,12.99,101.07,2,11,54.0,10,1,29000.0,2,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,2016,3,1,1977,8


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 49 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       800000 non-null  int64  
 1   loanAmnt                 800000 non-null  float64
 2   term                     800000 non-null  int64  
 3   interestRate             800000 non-null  float64
 4   installment              800000 non-null  float64
 5   grade                    800000 non-null  int64  
 6   subGrade                 800000 non-null  int64  
 7   employmentTitle          800000 non-null  float64
 8   employmentLength         800000 non-null  int64  
 9   homeOwnership            800000 non-null  int64  
 10  annualIncome             800000 non-null  float64
 11  verificationStatus       800000 non-null  int64  
 12  purpose                  800000 non-null  int64  
 13  postCode                 800000 non-null  float64
 14  regi

### (一)日期时间创造

In [15]:
def datetime_transform(df):
    from datetime import datetime
    datetime_list=[datetime(x,y,z) for x,y,z in zip(
        df['issue_year'],
        df['issue_month'],
        df['issue_day'])]
    datetime_list=pd.DataFrame(datetime_list)
    datetime_list.columns=['columns_']
    week_=datetime_list['columns_'].apply(lambda x:x.week)
    quarter_=datetime_list['columns_'].apply(lambda x:x.quarter)
    dayofyear_=datetime_list['columns_'].apply(lambda x:x.dayofyear)
    df['issue_week']=week_
    df['issue_quarter']=quarter_
    df['issue_dayofyear']=dayofyear_
    return df

In [16]:
train_data=datetime_transform(train_data)
test_data=datetime_transform(test_data)

In [17]:
train_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month,issue_week,issue_quarter,issue_dayofyear
0,0,35000.0,5,19.52,917.97,4,21,320.0,2,2,110000.0,2,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,2001,8,27,3,182
1,1,18000.0,5,18.49,461.90,3,16,219843.0,5,0,46000.0,2,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,1723.0,1.0,0.0,3.0,4.0,4.0,10.0,5.0,4.0,6.0,11.0,4.0,13.0,0.0,0.0,0.0,1.0,2012,8,1,2002,5,31,3,214
2,2,12000.0,5,16.99,298.17,3,17,31698.0,8,0,74000.0,2,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,1,2006,5,40,4,274
3,3,11000.0,3,7.26,340.96,0,3,46854.0,10,1,118000.0,1,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,1,1999,5,31,3,213
4,4,3000.0,3,12.99,101.07,2,11,54.0,10,1,29000.0,2,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,2016,3,1,1977,8,9,1,61


In [18]:
test_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month,issue_week,issue_quarter,issue_dayofyear
0,800000,14000.0,3,10.99,458.28,1,7,7027.0,10,0,80000.0,0,0,163.0,21,10.56,1.0,715.0,719.0,17.0,0.0,0.0,9846.0,30.7,29.0,0,0,0.0,1.0,1.0,4.0,6.0,6.0,6.0,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0,2014,7,1,1974,11,27,3,182
1,800001,20000.0,5,14.65,472.14,2,14,60426.0,10,0,50000.0,0,2,235.0,8,21.40,2.0,670.0,674.0,5.0,0.0,0.0,8946.0,56.6,14.0,0,0,5.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0,2015,7,1,2001,7,27,3,182
2,800002,12000.0,3,19.99,445.91,3,18,23547.0,2,1,60000.0,2,0,526.0,20,33.50,0.0,710.0,714.0,12.0,0.0,0.0,970.0,17.6,43.0,1,0,0.0,1.0,0.0,1.0,4.0,4.0,1.0,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0,2016,10,1,2006,8,39,4,275
3,800003,17500.0,5,14.31,410.02,2,13,636.0,4,0,37000.0,1,4,248.0,11,13.95,0.0,685.0,689.0,10.0,1.0,1.0,10249.0,52.3,18.0,0,0,4.0,1.0,0.0,2.0,2.0,2.0,4.0,7.0,2.0,8.0,14.0,2.0,10.0,0.0,0.0,0.0,3.0,2014,11,1,2002,7,44,4,305
4,800004,35000.0,3,17.09,1249.42,3,15,368446.0,0,1,80000.0,1,0,115.0,8,24.97,0.0,685.0,689.0,19.0,0.0,0.0,33199.0,35.6,22.0,0,0,0.0,1.0,0.0,8.0,11.0,11.0,9.0,11.0,3.0,16.0,18.0,11.0,19.0,0.0,0.0,0.0,1.0,2017,10,1,2000,12,39,4,274


### (二)连续值进行数据分箱

In [19]:
#设置数据分箱的函数
'''
    df:输入的数据集
    col_name:需要分箱的字段
    k:分箱的数量，默认值为5
'''
def categorical_data_cut(df,col_name,k=5):
    cut_=pd.cut(x=df[col_name],bins=k,labels=range(1,k+1))
    df["cut_{}_{}".format(k,col_name)]=cut_
    return df

In [20]:
#调用分箱函数
#贷款金额loanAmnt分箱
train_data=categorical_data_cut(train_data,'loanAmnt',k=5)
test_data=categorical_data_cut(test_data,'loanAmnt',k=5)

#贷款利率分箱interestRate
train_data=categorical_data_cut(train_data,'interestRate',k=6)
test_data=categorical_data_cut(test_data,'interestRate',k=6)

#分期金额分箱installment
train_data=categorical_data_cut(train_data,'installment',k=6)
test_data=categorical_data_cut(test_data,'installment',k=6)

#年收入分箱annualIncome
train_data=categorical_data_cut(train_data,'annualIncome',k=6)
test_data=categorical_data_cut(test_data,'annualIncome',k=6)

#债务收入比
train_data=categorical_data_cut(train_data,'dti',k=5)
test_data=categorical_data_cut(test_data,'dti',k=5)

#ficoRangeLow and ficoRangeHigh
train_data=categorical_data_cut(train_data,'ficoRangeLow',k=5)
train_data=categorical_data_cut(train_data,'ficoRangeHigh',k=5)

test_data=categorical_data_cut(test_data,'ficoRangeLow',k=5)
test_data=categorical_data_cut(test_data,'ficoRangeHigh',k=5)

#revolBal
train_data=categorical_data_cut(train_data,'revolBal',k=5)
test_data=categorical_data_cut(test_data,'revolBal',k=5)

#revolUtil
train_data=categorical_data_cut(train_data,'revolUtil',k=5)
test_data=categorical_data_cut(test_data,'revolUtil',k=5)

#totalAcc
train_data=categorical_data_cut(train_data,'totalAcc',k=5)
test_data=categorical_data_cut(test_data,'totalAcc',k=5)

In [29]:
train_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month,issue_week,issue_quarter,issue_dayofyear,cut_5_loanAmnt,cut_6_interestRate,cut_6_installment,cut_6_annualIncome,cut_5_dti,cut_5_ficoRangeLow,cut_5_ficoRangeHigh,cut_5_revolBal,cut_5_revolUtil,cut_5_totalAcc,cut_5_loanAmnt_count,cut_6_interestRate_count,cut_6_installment_count,cut_6_annualIncome_count,cut_5_dti_count,cut_5_ficoRangeLow_count,cut_5_ficoRangeHigh_count,cut_5_revolBal_count,cut_5_revolUtil_count,cut_5_totalAcc_count
0,0,35000.0,5,19.52,917.97,4,21,320.0,2,2,110000.0,2,1,137.0,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,1.0,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2014,7,1,2001,8,27,3,182,5,4,4,1,1,3,3,1,1,1,41273,82330,45596,799940,799889,119896,119896,799940,799997,645494
1,1,18000.0,5,18.49,461.90,3,16,219843.0,5,0,46000.0,2,0,156.0,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,1723.0,1.0,0.0,3.0,4.0,4.0,10.0,5.0,4.0,6.0,11.0,4.0,13.0,0.0,0.0,0.0,1.0,2012,8,1,2002,5,31,3,214,3,4,2,1,1,2,2,1,1,1,151884,82330,325673,799940,799889,422838,422838,799940,799997,645494
2,2,12000.0,5,16.99,298.17,3,17,31698.0,8,0,74000.0,2,0,337.0,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,0.0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,2015,10,1,2006,5,40,4,274,2,3,1,1,1,2,2,1,1,1,294964,205493,275735,799940,799889,422838,422838,799940,799997,645494
3,3,11000.0,3,7.26,340.96,0,3,46854.0,10,1,118000.0,1,4,148.0,11,17.21,0.0,685.0,689.0,9.0,0.0,0.0,9948.0,52.6,28.0,1,0,4.0,1.0,6.0,4.0,6.0,6.0,4.0,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0,2015,8,1,1999,5,31,3,213,2,1,2,1,1,2,2,1,1,1,294964,194051,325673,799940,799889,422838,422838,799940,799997,645494
4,4,3000.0,3,12.99,101.07,2,11,54.0,10,1,29000.0,2,10,301.0,21,32.16,0.0,690.0,694.0,12.0,0.0,0.0,2942.0,32.0,27.0,0,0,11.0,1.0,1.0,2.0,7.0,7.0,2.0,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0,2016,3,1,1977,8,9,1,61,1,2,1,1,1,2,2,1,1,1,235293,281035,275735,799940,799889,422838,422838,799940,799997,645494


In [30]:
test_data.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issue_year,issue_month,issue_day,earliesCreditLine_year,earliesCreditLine_month,issue_week,issue_quarter,issue_dayofyear,cut_5_loanAmnt,cut_6_interestRate,cut_6_installment,cut_6_annualIncome,cut_5_dti,cut_5_ficoRangeLow,cut_5_ficoRangeHigh,cut_5_revolBal,cut_5_revolUtil,cut_5_totalAcc,cut_5_loanAmnt_count,cut_6_interestRate_count,cut_6_installment_count,cut_6_annualIncome_count,cut_5_dti_count,cut_5_ficoRangeLow_count,cut_5_ficoRangeHigh_count,cut_5_revolBal_count,cut_5_revolUtil_count,cut_5_totalAcc_count
0,800000,14000.0,3,10.99,458.28,1,7,7027.0,10,0,80000.0,0,0,163.0,21,10.56,1.0,715.0,719.0,17.0,0.0,0.0,9846.0,30.7,29.0,0,0,0.0,1.0,1.0,4.0,6.0,6.0,6.0,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0,2014,7,1,1974,11,27,3,182,2,2,2,1,1,3,3,1,1,2,73965,70267,81657,199976,199972,37281,37281,199906,156132,66822
1,800001,20000.0,5,14.65,472.14,2,14,60426.0,10,0,50000.0,0,2,235.0,8,21.40,2.0,670.0,674.0,5.0,0.0,0.0,8946.0,56.6,14.0,0,0,5.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0,2015,7,1,2001,7,27,3,182,3,3,2,1,1,2,2,1,1,1,37664,51312,81657,199976,199972,116086,116086,199906,156132,127879
2,800002,12000.0,3,19.99,445.91,3,18,23547.0,2,1,60000.0,2,0,526.0,20,33.50,0.0,710.0,714.0,12.0,0.0,0.0,970.0,17.6,43.0,1,0,0.0,1.0,0.0,1.0,4.0,4.0,1.0,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0,2016,10,1,2006,8,39,4,275,2,4,2,1,1,2,2,1,1,2,73965,20757,81657,199976,199972,116086,116086,199906,156132,66822
3,800003,17500.0,5,14.31,410.02,2,13,636.0,4,0,37000.0,1,4,248.0,11,13.95,0.0,685.0,689.0,10.0,1.0,1.0,10249.0,52.3,18.0,0,0,4.0,1.0,0.0,2.0,2.0,2.0,4.0,7.0,2.0,8.0,14.0,2.0,10.0,0.0,0.0,0.0,3.0,2014,11,1,2002,7,44,4,305,3,3,2,1,1,2,2,1,1,1,37664,51312,81657,199976,199972,116086,116086,199906,156132,127879
4,800004,35000.0,3,17.09,1249.42,3,15,368446.0,0,1,80000.0,1,0,115.0,8,24.97,0.0,685.0,689.0,19.0,0.0,0.0,33199.0,35.6,22.0,0,0,0.0,1.0,0.0,8.0,11.0,11.0,9.0,11.0,3.0,16.0,18.0,11.0,19.0,0.0,0.0,0.0,1.0,2017,10,1,2000,12,39,4,274,5,3,5,1,1,2,2,1,1,1,10543,51312,4165,199976,199972,116086,116086,199906,156132,127879


In [23]:
train_data.shape

(800000, 62)

In [24]:
test_data.shape

(200000, 62)

### （三）对分箱后的字段做计数处理

In [25]:
def bins_count(df,col_name):
    df_data=df[[col_name]]
    df_data["{}_count".format(col_name)]=1
    df_groupby=df_data.groupby(col_name).agg('sum').\
        reset_index()
    df=df.merge(df_groupby,on=col_name,how='left')
    return df

In [26]:
#获取字段所在的列的信息,用字典的形式展示
column_list=list(train_data.columns)
col_list={index:label for index,label in enumerate(column_list)}

In [27]:
count_list=train_data.iloc[:,52:62]

In [28]:
#调用分箱统计函数
for col_name in count_list.columns:
    train_data=bins_count(train_data,col_name)
for col_name in count_list.columns:
    test_data=bins_count(test_data,col_name)

In [35]:
train_data.shape

(800000, 72)